In [9]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn as nn
from torchvision import datasets,transforms
import torch.nn.functional as F
import sys
sys.path.append("../code/")
from dataloader import APPLIANCE_ORDER, get_train_test
from sklearn.metrics import mean_absolute_error
import os
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('../code/')
%matplotlib inline
import itertools
from pathlib import Path

In [20]:
def onoff_error(pred, gt, threshold):
    abs_error = np.abs(pred-gt)
    error = [x for x in abs_error.reshape(1, -1).tolist()[0] if x >= threshold]
    
    return np.mean(error)

In [6]:
dataset = 5
gid = 0
srv = 1
c = 0
cmd = {}
cnn_tree_valid_pred = {}
num_iterations = 20000
lr = 0.01
p = 0
for fold_num in range(5):
    cnn_tree_valid_pred[fold_num] = {}
    for lr in [0.01]:
        cnn_tree_valid_pred[fold_num][lr] = {}
        for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw', 'residual'])):
            print(fold_num, lr, order)
#             if order[0] == 'hvac':
#                 continue
            
            cnn_tree_valid_pred[fold_num][lr][order] = {}


            o = "\', \'".join(str(x) for x in order)
            directory = "../code/baseline/cnn-tree/{}/{}/{}/20000/0.0/".format(dataset, fold_num, lr)
            filename = "valid-pred-[\'{}\'].npy".format(o)

            full_path = directory + filename
            my_file = Path(full_path)
            if not my_file.exists():
                o = " ".join(str(x) for x in order)
                line = ("CUDA_VISIBLE_DEVICES={} python cnn-tree.py 3 {} {} 0 {} {} &".format(gid, lr, num_iterations, fold_num, o))
#                 print(line)

                c += 1
                if c%1 == 0:
                    gid+=1
                if gid == 4:
                    gid = 0
                    srv += 1
            else:
                k = np.load(full_path).item()
                for it in range(1000, 20001, 1000):
                    cnn_tree_valid_pred[fold_num][lr][order][it] = {}
                    for idx, appliance in enumerate(order):
                        cnn_tree_valid_pred[fold_num][lr][order][it][appliance] = k[it][idx]

print(c)                               

0 0.01 ('hvac', 'fridge', 'dr', 'dw', 'mw', 'residual')
0 0.01 ('hvac', 'fridge', 'dr', 'dw', 'residual', 'mw')
0 0.01 ('hvac', 'fridge', 'dr', 'mw', 'dw', 'residual')
0 0.01 ('hvac', 'fridge', 'dr', 'mw', 'residual', 'dw')
0 0.01 ('hvac', 'fridge', 'dr', 'residual', 'dw', 'mw')
0 0.01 ('hvac', 'fridge', 'dr', 'residual', 'mw', 'dw')
0 0.01 ('hvac', 'fridge', 'dw', 'dr', 'mw', 'residual')
0 0.01 ('hvac', 'fridge', 'dw', 'dr', 'residual', 'mw')
0 0.01 ('hvac', 'fridge', 'dw', 'mw', 'dr', 'residual')
0 0.01 ('hvac', 'fridge', 'dw', 'mw', 'residual', 'dr')
0 0.01 ('hvac', 'fridge', 'dw', 'residual', 'dr', 'mw')
0 0.01 ('hvac', 'fridge', 'dw', 'residual', 'mw', 'dr')
0 0.01 ('hvac', 'fridge', 'mw', 'dr', 'dw', 'residual')
0 0.01 ('hvac', 'fridge', 'mw', 'dr', 'residual', 'dw')
0 0.01 ('hvac', 'fridge', 'mw', 'dw', 'dr', 'residual')
0 0.01 ('hvac', 'fridge', 'mw', 'dw', 'residual', 'dr')
0 0.01 ('hvac', 'fridge', 'mw', 'residual', 'dr', 'dw')
0 0.01 ('hvac', 'fridge', 'mw', 'residual', 'dw'

0 0.01 ('fridge', 'dr', 'hvac', 'residual', 'dw', 'mw')
0 0.01 ('fridge', 'dr', 'hvac', 'residual', 'mw', 'dw')
0 0.01 ('fridge', 'dr', 'dw', 'hvac', 'mw', 'residual')
0 0.01 ('fridge', 'dr', 'dw', 'hvac', 'residual', 'mw')
0 0.01 ('fridge', 'dr', 'dw', 'mw', 'hvac', 'residual')
0 0.01 ('fridge', 'dr', 'dw', 'mw', 'residual', 'hvac')
0 0.01 ('fridge', 'dr', 'dw', 'residual', 'hvac', 'mw')
0 0.01 ('fridge', 'dr', 'dw', 'residual', 'mw', 'hvac')
0 0.01 ('fridge', 'dr', 'mw', 'hvac', 'dw', 'residual')
0 0.01 ('fridge', 'dr', 'mw', 'hvac', 'residual', 'dw')
0 0.01 ('fridge', 'dr', 'mw', 'dw', 'hvac', 'residual')
0 0.01 ('fridge', 'dr', 'mw', 'dw', 'residual', 'hvac')
0 0.01 ('fridge', 'dr', 'mw', 'residual', 'hvac', 'dw')
0 0.01 ('fridge', 'dr', 'mw', 'residual', 'dw', 'hvac')
0 0.01 ('fridge', 'dr', 'residual', 'hvac', 'dw', 'mw')
0 0.01 ('fridge', 'dr', 'residual', 'hvac', 'mw', 'dw')
0 0.01 ('fridge', 'dr', 'residual', 'dw', 'hvac', 'mw')
0 0.01 ('fridge', 'dr', 'residual', 'dw', 'mw', 

0 0.01 ('dr', 'mw', 'fridge', 'dw', 'residual', 'hvac')
0 0.01 ('dr', 'mw', 'fridge', 'residual', 'hvac', 'dw')
0 0.01 ('dr', 'mw', 'fridge', 'residual', 'dw', 'hvac')
0 0.01 ('dr', 'mw', 'dw', 'hvac', 'fridge', 'residual')
0 0.01 ('dr', 'mw', 'dw', 'hvac', 'residual', 'fridge')
0 0.01 ('dr', 'mw', 'dw', 'fridge', 'hvac', 'residual')
0 0.01 ('dr', 'mw', 'dw', 'fridge', 'residual', 'hvac')
0 0.01 ('dr', 'mw', 'dw', 'residual', 'hvac', 'fridge')
0 0.01 ('dr', 'mw', 'dw', 'residual', 'fridge', 'hvac')
0 0.01 ('dr', 'mw', 'residual', 'hvac', 'fridge', 'dw')
0 0.01 ('dr', 'mw', 'residual', 'hvac', 'dw', 'fridge')
0 0.01 ('dr', 'mw', 'residual', 'fridge', 'hvac', 'dw')
0 0.01 ('dr', 'mw', 'residual', 'fridge', 'dw', 'hvac')
0 0.01 ('dr', 'mw', 'residual', 'dw', 'hvac', 'fridge')
0 0.01 ('dr', 'mw', 'residual', 'dw', 'fridge', 'hvac')
0 0.01 ('dr', 'residual', 'hvac', 'fridge', 'dw', 'mw')
0 0.01 ('dr', 'residual', 'hvac', 'fridge', 'mw', 'dw')
0 0.01 ('dr', 'residual', 'hvac', 'dw', 'fridge'

0 0.01 ('mw', 'hvac', 'dr', 'residual', 'fridge', 'dw')
0 0.01 ('mw', 'hvac', 'dr', 'residual', 'dw', 'fridge')
0 0.01 ('mw', 'hvac', 'dw', 'fridge', 'dr', 'residual')
0 0.01 ('mw', 'hvac', 'dw', 'fridge', 'residual', 'dr')
0 0.01 ('mw', 'hvac', 'dw', 'dr', 'fridge', 'residual')
0 0.01 ('mw', 'hvac', 'dw', 'dr', 'residual', 'fridge')
0 0.01 ('mw', 'hvac', 'dw', 'residual', 'fridge', 'dr')
0 0.01 ('mw', 'hvac', 'dw', 'residual', 'dr', 'fridge')
0 0.01 ('mw', 'hvac', 'residual', 'fridge', 'dr', 'dw')
0 0.01 ('mw', 'hvac', 'residual', 'fridge', 'dw', 'dr')
0 0.01 ('mw', 'hvac', 'residual', 'dr', 'fridge', 'dw')
0 0.01 ('mw', 'hvac', 'residual', 'dr', 'dw', 'fridge')
0 0.01 ('mw', 'hvac', 'residual', 'dw', 'fridge', 'dr')
0 0.01 ('mw', 'hvac', 'residual', 'dw', 'dr', 'fridge')
0 0.01 ('mw', 'fridge', 'hvac', 'dr', 'dw', 'residual')
0 0.01 ('mw', 'fridge', 'hvac', 'dr', 'residual', 'dw')
0 0.01 ('mw', 'fridge', 'hvac', 'dw', 'dr', 'residual')
0 0.01 ('mw', 'fridge', 'hvac', 'dw', 'residual'

0 0.01 ('residual', 'fridge', 'dw', 'hvac', 'mw', 'dr')
0 0.01 ('residual', 'fridge', 'dw', 'dr', 'hvac', 'mw')
0 0.01 ('residual', 'fridge', 'dw', 'dr', 'mw', 'hvac')
0 0.01 ('residual', 'fridge', 'dw', 'mw', 'hvac', 'dr')
0 0.01 ('residual', 'fridge', 'dw', 'mw', 'dr', 'hvac')
0 0.01 ('residual', 'fridge', 'mw', 'hvac', 'dr', 'dw')
0 0.01 ('residual', 'fridge', 'mw', 'hvac', 'dw', 'dr')
0 0.01 ('residual', 'fridge', 'mw', 'dr', 'hvac', 'dw')
0 0.01 ('residual', 'fridge', 'mw', 'dr', 'dw', 'hvac')
0 0.01 ('residual', 'fridge', 'mw', 'dw', 'hvac', 'dr')
0 0.01 ('residual', 'fridge', 'mw', 'dw', 'dr', 'hvac')
0 0.01 ('residual', 'dr', 'hvac', 'fridge', 'dw', 'mw')
0 0.01 ('residual', 'dr', 'hvac', 'fridge', 'mw', 'dw')
0 0.01 ('residual', 'dr', 'hvac', 'dw', 'fridge', 'mw')
0 0.01 ('residual', 'dr', 'hvac', 'dw', 'mw', 'fridge')
0 0.01 ('residual', 'dr', 'hvac', 'mw', 'fridge', 'dw')
0 0.01 ('residual', 'dr', 'hvac', 'mw', 'dw', 'fridge')
0 0.01 ('residual', 'dr', 'fridge', 'hvac', 'dw'

...

4 0.01 ('hvac', 'fridge', 'dw', 'residual', 'dr', 'mw')
4 0.01 ('hvac', 'fridge', 'dw', 'residual', 'mw', 'dr')
4 0.01 ('hvac', 'fridge', 'mw', 'dr', 'dw', 'residual')
4 0.01 ('hvac', 'fridge', 'mw', 'dr', 'residual', 'dw')
4 0.01 ('hvac', 'fridge', 'mw', 'dw', 'dr', 'residual')
4 0.01 ('hvac', 'fridge', 'mw', 'dw', 'residual', 'dr')
4 0.01 ('hvac', 'fridge', 'mw', 'residual', 'dr', 'dw')
4 0.01 ('hvac', 'fridge', 'mw', 'residual', 'dw', 'dr')
4 0.01 ('hvac', 'fridge', 'residual', 'dr', 'dw', 'mw')
4 0.01 ('hvac', 'fridge', 'residual', 'dr', 'mw', 'dw')
4 0.01 ('hvac', 'fridge', 'residual', 'dw', 'dr', 'mw')
4 0.01 ('hvac', 'fridge', 'residual', 'dw', 'mw', 'dr')
4 0.01 ('hvac', 'fridge', 'residual', 'mw', 'dr', 'dw')
4 0.01 ('hvac', 'fridge', 'residual', 'mw', 'dw', 'dr')
4 0.01 ('hvac', 'dr', 'fridge', 'dw', 'mw', 'residual')
4 0.01 ('hvac', 'dr', 'fridge', 'dw', 'residual', 'mw')
4 0.01 ('hvac', 'dr', 'fridge', 'mw', 'dw', 'residual')
4 0.01 ('hvac', 'dr', 'fridge', 'mw', 'residual'

4 0.01 ('fridge', 'dr', 'mw', 'hvac', 'residual', 'dw')
4 0.01 ('fridge', 'dr', 'mw', 'dw', 'hvac', 'residual')
4 0.01 ('fridge', 'dr', 'mw', 'dw', 'residual', 'hvac')
4 0.01 ('fridge', 'dr', 'mw', 'residual', 'hvac', 'dw')
4 0.01 ('fridge', 'dr', 'mw', 'residual', 'dw', 'hvac')
4 0.01 ('fridge', 'dr', 'residual', 'hvac', 'dw', 'mw')
4 0.01 ('fridge', 'dr', 'residual', 'hvac', 'mw', 'dw')
4 0.01 ('fridge', 'dr', 'residual', 'dw', 'hvac', 'mw')
4 0.01 ('fridge', 'dr', 'residual', 'dw', 'mw', 'hvac')
4 0.01 ('fridge', 'dr', 'residual', 'mw', 'hvac', 'dw')
4 0.01 ('fridge', 'dr', 'residual', 'mw', 'dw', 'hvac')
4 0.01 ('fridge', 'dw', 'hvac', 'dr', 'mw', 'residual')
4 0.01 ('fridge', 'dw', 'hvac', 'dr', 'residual', 'mw')
4 0.01 ('fridge', 'dw', 'hvac', 'mw', 'dr', 'residual')
4 0.01 ('fridge', 'dw', 'hvac', 'mw', 'residual', 'dr')
4 0.01 ('fridge', 'dw', 'hvac', 'residual', 'dr', 'mw')
4 0.01 ('fridge', 'dw', 'hvac', 'residual', 'mw', 'dr')
4 0.01 ('fridge', 'dw', 'dr', 'hvac', 'mw', 'res

4 0.01 ('dr', 'dw', 'mw', 'residual', 'hvac', 'fridge')
4 0.01 ('dr', 'dw', 'mw', 'residual', 'fridge', 'hvac')
4 0.01 ('dr', 'dw', 'residual', 'hvac', 'fridge', 'mw')
4 0.01 ('dr', 'dw', 'residual', 'hvac', 'mw', 'fridge')
4 0.01 ('dr', 'dw', 'residual', 'fridge', 'hvac', 'mw')
4 0.01 ('dr', 'dw', 'residual', 'fridge', 'mw', 'hvac')
4 0.01 ('dr', 'dw', 'residual', 'mw', 'hvac', 'fridge')
4 0.01 ('dr', 'dw', 'residual', 'mw', 'fridge', 'hvac')
4 0.01 ('dr', 'mw', 'hvac', 'fridge', 'dw', 'residual')
4 0.01 ('dr', 'mw', 'hvac', 'fridge', 'residual', 'dw')
4 0.01 ('dr', 'mw', 'hvac', 'dw', 'fridge', 'residual')
4 0.01 ('dr', 'mw', 'hvac', 'dw', 'residual', 'fridge')
4 0.01 ('dr', 'mw', 'hvac', 'residual', 'fridge', 'dw')
4 0.01 ('dr', 'mw', 'hvac', 'residual', 'dw', 'fridge')
4 0.01 ('dr', 'mw', 'fridge', 'hvac', 'dw', 'residual')
4 0.01 ('dr', 'mw', 'fridge', 'hvac', 'residual', 'dw')
4 0.01 ('dr', 'mw', 'fridge', 'dw', 'hvac', 'residual')
4 0.01 ('dr', 'mw', 'fridge', 'dw', 'residual', 

4 0.01 ('dw', 'mw', 'residual', 'hvac', 'dr', 'fridge')
4 0.01 ('dw', 'mw', 'residual', 'fridge', 'hvac', 'dr')
4 0.01 ('dw', 'mw', 'residual', 'fridge', 'dr', 'hvac')
4 0.01 ('dw', 'mw', 'residual', 'dr', 'hvac', 'fridge')
4 0.01 ('dw', 'mw', 'residual', 'dr', 'fridge', 'hvac')
4 0.01 ('dw', 'residual', 'hvac', 'fridge', 'dr', 'mw')
4 0.01 ('dw', 'residual', 'hvac', 'fridge', 'mw', 'dr')
4 0.01 ('dw', 'residual', 'hvac', 'dr', 'fridge', 'mw')
4 0.01 ('dw', 'residual', 'hvac', 'dr', 'mw', 'fridge')
4 0.01 ('dw', 'residual', 'hvac', 'mw', 'fridge', 'dr')
4 0.01 ('dw', 'residual', 'hvac', 'mw', 'dr', 'fridge')
4 0.01 ('dw', 'residual', 'fridge', 'hvac', 'dr', 'mw')
4 0.01 ('dw', 'residual', 'fridge', 'hvac', 'mw', 'dr')
4 0.01 ('dw', 'residual', 'fridge', 'dr', 'hvac', 'mw')
4 0.01 ('dw', 'residual', 'fridge', 'dr', 'mw', 'hvac')
4 0.01 ('dw', 'residual', 'fridge', 'mw', 'hvac', 'dr')
4 0.01 ('dw', 'residual', 'fridge', 'mw', 'dr', 'hvac')
4 0.01 ('dw', 'residual', 'dr', 'hvac', 'fridge'

4 0.01 ('mw', 'residual', 'dw', 'dr', 'hvac', 'fridge')
4 0.01 ('mw', 'residual', 'dw', 'dr', 'fridge', 'hvac')
4 0.01 ('residual', 'hvac', 'fridge', 'dr', 'dw', 'mw')
4 0.01 ('residual', 'hvac', 'fridge', 'dr', 'mw', 'dw')
4 0.01 ('residual', 'hvac', 'fridge', 'dw', 'dr', 'mw')
4 0.01 ('residual', 'hvac', 'fridge', 'dw', 'mw', 'dr')
4 0.01 ('residual', 'hvac', 'fridge', 'mw', 'dr', 'dw')
4 0.01 ('residual', 'hvac', 'fridge', 'mw', 'dw', 'dr')
4 0.01 ('residual', 'hvac', 'dr', 'fridge', 'dw', 'mw')
4 0.01 ('residual', 'hvac', 'dr', 'fridge', 'mw', 'dw')
4 0.01 ('residual', 'hvac', 'dr', 'dw', 'fridge', 'mw')
4 0.01 ('residual', 'hvac', 'dr', 'dw', 'mw', 'fridge')
4 0.01 ('residual', 'hvac', 'dr', 'mw', 'fridge', 'dw')
4 0.01 ('residual', 'hvac', 'dr', 'mw', 'dw', 'fridge')
4 0.01 ('residual', 'hvac', 'dw', 'fridge', 'dr', 'mw')
4 0.01 ('residual', 'hvac', 'dw', 'fridge', 'mw', 'dr')
4 0.01 ('residual', 'hvac', 'dw', 'dr', 'fridge', 'mw')
4 0.01 ('residual', 'hvac', 'dw', 'dr', 'mw', 'f

In [10]:
tensor = np.load("../2015-5appliances.numpy.npy")
dr = tensor[:, 3]

# calculate gt
test_gt = {}
valid_gt = {}
for fold_num in range(5):
    test_gt[fold_num] = {}
    valid_gt[fold_num] = {}
    train, test = get_train_test(1, 5, fold_num)
    valid = train[int(0.8*len(train)):].copy()
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        test_gt[fold_num][appliance] = test[:, idx+1]
        valid_gt[fold_num][appliance] = valid[:, idx+1]
        
threshold = {}
for appliance in ['dr', 'dw', 'mw']:
    sample_list = []
    for fold_num in range(5):
        sample_list = np.append(sample_list, [x for x in test_gt[fold_num][appliance].reshape(1, -1).tolist()[0] if x > 5])
    mean = np.mean(sample_list)
    print(appliance, mean)
    threshold[appliance] = 0.1*mean

dr 725.849570636
dw 285.780713239
mw 31.7082756317


In [23]:
cnn_tree_best_param = {}
for fold_num in range(5):
    cnn_tree_best_param[fold_num] = {}
    min_error = np.inf
    for lr in [0.01]:
        for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw', 'residual'])):

            for it in range(1000, 20001, 1000):
                error = 0
                try:
                    for idx, appliance in enumerate(order):
                        if appliance == 'residual':
                            continue
                        if appliance in ['hvac', 'fridge']:
                            error += mean_absolute_error(cnn_tree_valid_pred[fold_num][lr][order][it][appliance].reshape(-1, 24),
                                                        valid_gt[fold_num][appliance].reshape(-1, 24))
                        else:
                            error += onoff_error(cnn_tree_valid_pred[fold_num][lr][order][it][appliance].reshape(-1, 24),
                                                        valid_gt[fold_num][appliance].reshape(-1, 24), threshold[appliance])
                    if error < min_error:
                        min_error = error
                        cnn_tree_best_param[fold_num]['lr'] = lr
                        cnn_tree_best_param[fold_num]['order'] = order
                        cnn_tree_best_param[fold_num]['iters'] = it
                except:
                    continue
                
                    

In [24]:
best_cnn_tree = cnn_tree_best_param

In [25]:
# get prediction for cnn tree
cnn_tree_pred = {}
for fold_num in range(5):
    cnn_tree_pred[fold_num] = {}
        
    lr = best_cnn_tree[fold_num]['lr']
    iters = best_cnn_tree[fold_num]['iters']
    order = best_cnn_tree[fold_num]['order']

    o = "\', \'".join(str(x) for x in order)
    directory = "../code/baseline/cnn-tree/{}/{}/{}/20000/0.0/".format(dataset, fold_num,lr)
    filename = "test-pred-[\'{}\'].npy".format(o)

    full_path = directory + filename
    cnn_pred = np.asarray(np.load(full_path).item()[iters])
    
    for idx, appliance in enumerate(order):
        cnn_tree_pred[fold_num][appliance] = cnn_pred[idx].reshape(-1, 1, 112,24)
    

In [26]:
def calculate_error(pred, gt, threshold):
    error = {}
    overall = {}
    num_homes = {}
    
    # calculte number of homes in each fold
    for fold_num in range(5):
        num_homes[fold_num] = gt[fold_num]['hvac'].reshape(-1, 1, 112,24).shape[0]
    homes = pd.Series(num_homes).sum()
    
    # for Constantly On appliances: hvac and fridge
    for appliance in ['hvac', 'fridge']:
        overall[appliance] = 0
        error[appliance] = {}
        for fold_num in range(5):
            error[appliance][fold_num] = mean_absolute_error(pred[fold_num][appliance].reshape(-1, 24), 
                                                             gt[fold_num][appliance].reshape(-1, 24))
            overall[appliance] += error[appliance][fold_num]*num_homes[fold_num]
        overall[appliance] /= homes
        
    # for ON/OFF appliances: dryer, dishwasher and microwave
    for appliance in ['dr', 'dw', 'mw']:
        error[appliance] = {}
        overall[appliance] = 0                                                                
        for fold_num in range(5):
            error[appliance][fold_num] = onoff_error(pred[fold_num][appliance].reshape(-1, 24), 
                                                     gt[fold_num][appliance].reshape(-1, 24), threshold[appliance])
            overall[appliance] += error[appliance][fold_num]*num_homes[fold_num]
        overall[appliance] /= homes
    
    
    return error, overall

In [30]:
pd.Series(calculate_error(cnn_tree_pred, test_gt, threshold)[1])

dr        695.579196
dw        218.650229
fridge     36.066408
hvac      397.938498
mw         23.917026
dtype: float64